# Timoschenko beam
We discuss the solutions of the Timoshenko beam for a uniformly distributed body force and a shear force. The equations for a given body force $f$, neglecting material parameters, read
\begin{align*}
&-\frac{1}{12}\beta^{\prime\prime}-\frac{\kappa G}{t^2}(w^\prime-\beta) = 0,\\
&-\frac{\kappa G}{t^2}(w^\prime-\beta)^\prime=f,
\end{align*}
where $t$ is the thickness, $G=E/(2(1+\nu))=0.5$ the shearing modulus, and $\kappa=5/6$ the shear correction factor.

Clamped boundaries are defined by $w=\beta=0$. Defining the shear force $Q=\frac{\kappa G}{t^2}(w^\prime-\beta)$ and bending moment $M=\beta^\prime$, free-boundaries are characterized by $Q=M=0$. A shear force can be prescribed by $Q=f$.

In [ ]:
from ngsolve import *
from ngsolve.meshes import Make1DMesh
from ngsolve.webgui import Draw
from sympy import symbols, simplify, solve, diff, lambdify

# use material parameters such that equations simplify
mu, lam = 0.5, 0  # Lame parameter
nu = lam/(2*(lam+mu)) # Possion ratio (=0)
E = mu*(3*lam+2*mu)/(lam+mu) # Young's modulus (=2*mu=1)
k = 5/6           # shear correction factor
G = E/(2*(1+nu))  # shearing modulus (=mu=0.5)

t = 0.01
alpha = 1/(12*k*G)

## Uniform distributed load

For a uniformly distributed load the exact solution of the Timoshenko beam is of the form
\begin{align*}
w(x) &= \frac{q}{2}x^4-\frac{c_1}{6}x^3-\frac{1}{2}(c_2+12\alpha\,t^2q)x^2+c_3x+c_4,\\
\beta(x) &=2qx^3-\frac{c_1}{2}x^2-c_2x-\alpha\,t^2c_1+c_3,
\end{align*}
where the constants $c_1$, $c_2$, $c_3$, $c_4$ are determined by the boundary conditions.

In [ ]:
# uniform distributed load
q = -0.1

X, C1, C2, C3, C4 = symbols("x C1 C2 C3 C4")
w_ex = q/2*X**4-C1/6*X**3-0.5*(C2+alpha*t**2*q*12)*X**2+C3*X+C4
w_diff_ex = 2*q*X**3-C1/2*X**2-(C2+alpha*t**2*q*12)*X+C3
beta_ex = 2*q*X**3-C1/2*X**2-C2*X-alpha*t**2*C1+C3
beta_diff_ex = q*12/2*X**2-C1*X-C2
if False: #check if the equations are fulfilled
    test_eq1 = simplify(-1/12*diff(beta_diff_ex,X)-k*G/t**2*(diff(w_ex,X)-beta_ex))
    print("test eq1:", test_eq1)
    test_eq2 = simplify(-k*G/t**2*(diff(w_diff_ex,X)-beta_diff_ex)-q)
    print("test eq2:", test_eq2)

# solve for boundary conditions: clamped at left, free at right boundary
solve_bc = solve((w_ex.subs(X,0), beta_ex.subs(X,0), k*G/t**2*(w_diff_ex-beta_ex).subs(X,1),\
                beta_diff_ex.subs(X,1)), C1, C2, C3, C4)
c1,c2,c3,c4 = solve_bc[C1], solve_bc[C2], solve_bc[C3], solve_bc[C4]

# generate CoefficientFunctions
w_ex_func = lambdify((X), w_ex.subs([(C1,c1),(C2,c2),(C3,c3),(C4,c4)]))
cf_w_ex = w_ex_func(x)
beta_ex_func = lambdify((X), beta_ex.subs([(C1,c1),(C2,c2),(C3,c3),(C4,c4)]))
cf_beta_ex = beta_ex_func(x)

# Test boundary conditions
print("test: w(0) =        ", w_ex.subs([(X,0),(C1,c1),(C2,c2),(C3,c3),(C4,c4)]))
print("test: (w'-beta)(1) =", 1/t**2*(w_diff_ex-beta_ex).subs([(X,1),(C1,c1),(C2,c2),(C3,c3),(C4,c4)]))
print("test: beta(0) =     ", beta_ex.subs([(X,0),(C1,c1),(C2,c2),(C3,c3),(C4,c4)]))
print("test: beta'(1) =    ", beta_diff_ex.subs([(X,1),(C1,c1),(C2,c2),(C3,c3),(C4,c4)]))

print("test: w(1) =        ", w_ex.subs([(X,1),(C1,c1),(C2,c2),(C3,c3),(C4,c4)]))

In [ ]:
mesh = Make1DMesh(10)
Draw(cf_w_ex, mesh, deformation=CF( (0, cf_w_ex, 0) ))
Draw(cf_beta_ex, mesh, deformation=CF( (0, cf_beta_ex, 0) ))

Solve the Timoshenko beam with Lagrange elements for the vertical deflections $w$ and the (linearized) rotation $\beta$ as unknowns. To avoid shear locking either quadratic elements, order=2, or number of mesh elements of order $1/t$, i.e. $h\approx t$, must be considered. Otherwise the solution tends to be zero.

In [ ]:
def SolveTB(order, mesh, t=t, draw=False):
    fes = H1(mesh, order=order, dirichlet="left")*H1(mesh, order=order, dirichlet="left") 
    (w,beta),(dw,dbeta) = fes.TnT()
    
    a = BilinearForm(fes, symmetric=True)
    a += 1/12*grad(beta)*grad(dbeta)*dx + k*G/t**2*(grad(w)-beta)*(grad(dw)-dbeta)*dx
    
    f = LinearForm(fes)
    f += q*dw*dx
    
    gfsol = GridFunction(fes)
    
    a.Assemble()
    f.Assemble()
    gfsol.vec.data = a.mat.Inverse(fes.FreeDofs(),inverse="sparsecholesky")*f.vec
    gfw, gfbeta = gfsol.components
    
    if draw:
        Draw(gfw,mesh, deformation=CF((0, gfw, 0)))
        Draw(gfbeta,mesh, deformation=CF((0, gfbeta, 0)))
    
    # relative errors
    errW = sqrt(Integrate((gfw-cf_w_ex)**2+(Grad(gfw)-cf_w_ex.Diff(x))**2,mesh))/sqrt(Integrate(cf_w_ex**2+cf_w_ex.Diff(x)**2,mesh))
    errbeta = sqrt(Integrate((gfbeta-cf_beta_ex)**2+(Grad(gfbeta)-cf_beta_ex.Diff(x))**2,mesh))/sqrt(Integrate(cf_beta_ex**2+cf_beta_ex.Diff(x)**2,mesh))

    return (gfw(mesh(1,0,0)), errW, errbeta)

# either order=2, or Make1DMesh with at least 100 elements
tip_deflection, errW, errSigma = SolveTB(order=1, mesh=Make1DMesh(100), t=t, draw=True)
print(f"{tip_deflection=}, {errW=}, {errSigma=}")

## Shear force

For a shear force the exact solution of the Timoshenko beam is of the form
\begin{align*}
w(x) &= -\frac{c_1}{6}x^3-\frac{c_2}{2}x^2 + c_3x+c_4,\\
\beta(x) &=-\frac{c_1}{2}x^2-c_2x-\alpha\,t^2c_1+c_3,
\end{align*}
where the constants $c_1$, $c_2$, $c_3$, $c_4$ are determined by the boundary conditions.

In [ ]:
# shear force
q = -0.1

w_ex = -C1/6*X**3-C2/2*X**2 + C3*X+C4
w_diff_ex = -C1/2*X**2-C2*X + C3
beta_ex = -C1/2*X**2-C2*X-alpha*t**2*C1+C3
beta_diff_ex = -C1*X-C2
if False:
    test_eq1 = simplify(-1/12*diff(beta_diff_ex,X)-k*G/t**2*(diff(w_ex,X)-beta_ex))
    print("test eq1:", test_eq1)
    test_eq2 = simplify(-k*G/t**2*(diff(w_diff_ex,X)-beta_diff_ex))
    print("test eq2:", test_eq2)

solve_bc = solve((w_ex.subs(X,0), beta_ex.subs(X,0), (k*G/t**2*(w_diff_ex-beta_ex)-q).subs(X,1), beta_diff_ex.subs(X,1)), C1, C2, C3, C4)
c1,c2,c3,c4 = solve_bc[C1], solve_bc[C2], solve_bc[C3], solve_bc[C4]

# generate CoefficientFunctions
w_ex_func = lambdify((X), w_ex.subs([(C1,c1),(C2,c2),(C3,c3),(C4,c4)]))
cf_w_ex = w_ex_func(x)
beta_ex_func = lambdify((X), beta_ex.subs([(C1,c1),(C2,c2),(C3,c3),(C4,c4)]))
cf_beta_ex = beta_ex_func(x)

print("test: w(0) =", w_ex.subs([(X,0),(C1,c1),(C2,c2),(C3,c3),(C4,c4)]))
print("test: k*G/t**2*(w'-beta)(1) =", k*G/t**2*(w_diff_ex-beta_ex).subs([(X,1),(C1,c1),(C2,c2),(C3,c3),(C4,c4)]))
print("test: beta(0) =", beta_ex.subs([(X,0),(C1,c1),(C2,c2),(C3,c3),(C4,c4)]))
print("test: beta'(1) =", diff(beta_ex,X).subs([(X,1),(C1,c1),(C2,c2),(C3,c3),(C4,c4)]))
print("test: w(1) =", w_ex.subs([(X,1),(C1,c1),(C2,c2),(C3,c3),(C4,c4)]))

In [ ]:
Draw(cf_w_ex, mesh, deformation=CF( (0, cf_w_ex, 0) ))
Draw(cf_beta_ex, mesh, deformation=CF( (0, cf_beta_ex, 0) ))

Solve the Timoshenko beam with Lagrange elements for the vertical deflections $w$ and the (linearized) rotation $\beta$ as unknowns.

In [ ]:
def SolveTB(order, mesh, t, reduced_integration=False, draw=False):
    fes = H1(mesh, order=order, dirichlet="left")*H1(mesh, order=order, dirichlet="left") 
    (w,beta),(dw,dbeta) = fes.TnT()
    
    a = BilinearForm(fes, symmetric=True)
    a += 1/12*grad(beta)*grad(dbeta)*dx + k*G/t**2*(grad(w)-beta)*(grad(dw)-dbeta)*dx(bonus_intorder=-reduced_integration)
    
    f = LinearForm(fes)
    f += q*dw*ds("right")
    
    gfsol = GridFunction(fes)
    
    a.Assemble()
    f.Assemble()
    gfsol.vec.data = a.mat.Inverse(fes.FreeDofs(),inverse="sparsecholesky")*f.vec
    gfw, gfbeta = gfsol.components
    
    if draw:
        Draw(gfw,mesh, deformation=CF((0, gfw, 0)))
        Draw(gfbeta,mesh, deformation=CF((0, gfbeta, 0)))
    
    errW = sqrt(Integrate((gfw-cf_w_ex)**2+(Grad(gfw)-cf_w_ex.Diff(x))**2,mesh))/sqrt(Integrate(cf_w_ex**2+cf_w_ex.Diff(x)**2,mesh))
    errbeta = sqrt(Integrate((gfbeta-cf_beta_ex)**2+(Grad(gfbeta)-cf_beta_ex.Diff(x))**2,mesh))/sqrt(Integrate(cf_beta_ex**2+cf_beta_ex.Diff(x)**2,mesh))
    return (gfw(mesh(1,0,0)), errW, errbeta)

# either order=2, or Make1DMesh with at least 100 elements
tip_deflection, errW, errSigma = SolveTB(order=1, mesh=Make1DMesh(100), t=t, draw=True)
print(f"{tip_deflection=}, {errW=}, {errSigma=}")